<a href="https://colab.research.google.com/github/AnerH13/Project-Hadzic-Rulisek/blob/main/Project_Hadzic_Rulisek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project - Bundesliga Table

---


## Importing necessary libraries

In [3]:
import pandas as pd
import requests

In [4]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


##  Downloading football rounds and then individual matches from Sofascore.com (This might take few minutes)

In [6]:
# Define a list to store all matches
all_matches = []

# Loop through rounds 1 to 34
for round_number in range(1, 35):
    url = f"https://www.sofascore.com/api/v1/unique-tournament/35/season/52608/events/round/{round_number}"
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        matches = response.json().get("events", [])
        all_matches.extend(matches)  # Use extend to flatten the list of matches
    else:
        print(f"Failed to retrieve matches for round {round_number}")

# Convert list of matches into a DataFrame
df_matches = pd.DataFrame(all_matches)

# Extract match IDs
df_matches_ids = df_matches["id"].tolist()

# Define headers for incidents API requests
headers = {
    "authority": "api.sofascore.com",
    "accept": "*/*",
    "accept-language": "cs-CZ,cs;q=0.9",
    "cache-control": "max-age=0",
    "if-none-match": 'W/"4a86ffe035"',
    "priority": "u=1, i",
    "referer": "https://www.sofascore.com/borussia-dortmund-tsg-hoffenheim/ubbsydb",
    "sec-ch-ua": '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "x-requested-with": "b75996"
}

# Define lists to store incidents

all_incidents = []

# Fetch incidents for each game
for game_id in df_matches_ids:
    response = requests.get(f"https://www.sofascore.com/api/v1/event/{game_id}/incidents", headers=headers)

    if response.status_code == 200:
        incidents = response.json()
        all_incidents.append(incidents)
    else:
        print(f"Failed to retrieve incidents for game ID {game_id}")



## Creating two functions, one that extracts important match events from raw data (downloaded above), second that creates readable data frame

In [9]:
def extract_events(game_data):
    events = []

    for incident in game_data['incidents']:
        if incident['incidentType'] == 'goal':
            goal_event = {
                'Event': 'Goal',
                'Minute': incident['time'],
                'Scorer': incident.get('player', {}).get('shortName', '-'),
                'Assist': incident.get('assist1', {}).get('shortName', '-'),
                'Team': 'Home' if incident.get('isHome', False) else 'Away'
            }
            events.append(goal_event)

        elif incident['incidentType'] == 'substitution':
            sub_event = {
                'Event': 'Substitution',
                'Minute': incident['time'],
                'Player In': incident.get('playerIn', {}).get('shortName', '-'),
                'Player Out': incident.get('playerOut', {}).get('shortName', '-'),
                'Team': 'Home' if incident.get('isHome', False) else 'Away'
            }
            events.append(sub_event)

        elif incident['incidentType'] == 'card':
            card_event = {
                'Event': 'Card',
                'Minute': incident['time'],
                'Player': incident.get('player', {}).get('shortName', '-'),
                'Reason': incident.get('reason', '-'),
                'Card Type': incident.get('incidentClass', '-'),  # 'yellow', 'red', etc.
                'Team': 'Home' if incident.get('isHome', False) else 'Away'
            }
            events.append(card_event)

        elif incident['incidentType'] == 'missed_penalty':
            missed_penalty_event = {
                'Event': 'Missed Penalty',
                'Minute': incident['time'],
                'Player': incident.get('player', {}).get('shortName', '-'),
                'Team': 'Home' if incident.get('isHome', False) else 'Away'
            }
            events.append(missed_penalty_event)

        elif incident['incidentType'] == 'var':
            var_event = {
                'Event': 'VAR',
                'Minute': incident['time'],
                'Player': incident.get('player', {}).get('shortName', '-'),
                'Reason': incident.get('reason', '-'),
                'Team': 'Home' if incident.get('isHome', False) else 'Away'
            }
            events.append(var_event)

    return events

def create_dataframe(events):
    # Create separate dataframes for home and away events
    df_home = pd.DataFrame([event for event in events if event['Team'] == 'Home'])
    df_away = pd.DataFrame([event for event in events if event['Team'] == 'Away'])

    # Rename columns to reflect home and away differentiation
    df_home.columns = [f'Home {col}' if col != 'Minute' else col for col in df_home.columns]
    df_away.columns = [f'Away {col}' if col != 'Minute' else col for col in df_away.columns]

    # Merge home and away dataframes on the Minute column
    # Concatenate the DataFrames
    df = pd.concat([df_home, df_away])

    # Sort the DataFrame by 'Minute'
    df = df.sort_values('Minute')

    # Reset the index if needed
    df = df.reset_index(drop=True)

    return df




## This is how one particular data frame with important match information looks like

In [10]:
create_dataframe(extract_events(all_incidents[2]))

,Home Event,Minute,Home Player In,Home Player Out,Home Team,Home Scorer,Home Assist,Home Player,Home Reason,Home Card Type,Away Event,Away Scorer,Away Assist,Away Team,Away Player In,Away Player Out,Away Player,Away Reason,Away Card Type
0,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Goal,K. Itakura,F. Honorat,Away,NaN,NaN,NaN,NaN,NaN
1,Card,23,NaN,NaN,Home,NaN,NaN,N. Dorsch,Foul,yellow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Goal,T. Čvančara,J. Weigl,Away,NaN,NaN,NaN,NaN,NaN
3,Goal,29,NaN,NaN,Home,E. Rexhbeçaj,S. Michel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Card,NaN,NaN,Away,NaN,NaN,T. Čvančara,Argument,yellow
5,NaN,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Goal,N. Ngoumou,J. Omlin,Away,NaN,NaN,NaN,NaN,NaN
6,Goal,41,NaN,NaN,Home,M. Bauer,E. Demirović,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Goal,45,NaN,NaN,Home,S. Michel,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Card,NaN,NaN,Away,NaN,NaN,F. Neuhaus,Foul,yellow
9,Substitution,58,D. D. Beljo,M. Berisha,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Creating data frame list with all matches

In [11]:
dataframes_list = []
for i in range(306):
    events = extract_events(all_incidents[i])
    df = create_dataframe(events)
    dataframes_list.append(df)

## Here we create games list, where we put a name to each individual match (this will be handy in further functions)

In [12]:
games_names = [ "BremenXBayern", "LeverkusenXLeipzig", "AugsburgXMgladbach", "HoffenheimXFreiburg", "StuttgartXBochum", "WolfsburgXHeidenheim", "DortmundXKoln", "Union_BerlinXMainz", "FrankfurtXDarmstadt",
    "LeipzigXStuttgart", "HeidenheimXHoffenheim", "KolnXWolfsburg", "DarmstadtXUnion_Berlin", "FreiburgXBremen", "BochumXDortmund", "MgladbachXLeverkusen", "MainzXFrankfurt", "BayernXAugsburg",
    "DortmundXHeidenheim", "LeverkusenXDarmstadt", "AugsburgXBochum", "BremenXMainz", "HoffenheimXWolfsburg", "StuttgartXFreiburg", "MgladbachXBayern", "FrankfurtXKoln", "Union_BerlinXLeipzig",
    "BayernXLeverkusen", "KolnXHoffenheim", "MainzXStuttgart", "LeipzigXAugsburg", "FreiburgXDortmund", "WolfsburgXUnion_Berlin", "BochumXFrankfurt", "HeidenheimXBremen", "DarmstadtXMgladbach",
    "StuttgartXDarmstadt","Union_BerlinXHoffenheim","DortmundXWolfsburg","MgladbachXLeipzig","AugsburgXMainz","BayernXBochum","BremenXKoln","LeverkusenXHeidenheim","FrankfurtXFreiburg",
    "HoffenheimXDortmund", "HeidenheimXUnion_Berlin", "KolnXStuttgart", "MainzXLeverkusen", "BochumXMgladbach", "WolfsburgXFrankfurt", "LeipzigXBayern", "DarmstadtXBremen", "FreiburgXAugsburg",
    "MgladbachXMainz", "DortmundXUnion_Berlin", "AugsburgXDarmstadt", "LeipzigXBochum", "StuttgartXWolfsburg", "BremenXHoffenheim", "LeverkusenXKoln", "BayernXFreiburg", "FrankfurtXHeidenheim",
    "DortmundXBremen", "Union_BerlinXStuttgart", "DarmstadtXLeipzig", "FreiburgXBochum", "HoffenheimXFrankfurt", "WolfsburgXLeverkusen", "MainzXBayern", "KolnXMgladbach", "HeidenheimXAugsburg",
    "BochumXMainz", "MgladbachXHeidenheim", "AugsburgXWolfsburg", "BayernXDarmstadt", "BremenXUnion_Berlin", "StuttgartXHoffenheim", "LeipzigXKoln", "FrankfurtXDortmund", "LeverkusenXFreiburg",
    "DarmstadtXBochum",  "KolnXAugsburg", "Union_BerlinXFrankfurt", "MainzXLeipzig",  "FreiburgXMgladbach",  "HoffenheimXLeverkusen",  "DortmundXBayern",  "WolfsburgXBremen",  "HeidenheimXStuttgart",
    "MgladbachXWolfsburg","DarmstadtXMainz", "AugsburgXHoffenheim",  "BayernXHeidenheim","StuttgartXDortmund","BochumXKoln","LeverkusenXUnion_Berlin","BremenXFrankfurt","LeipzigXFreiburg",
    "KolnXBayern","Union_BerlinXAugsburg","DortmundXMgladbach","FreiburgXDarmstadt","BremenXLeverkusen","WolfsburgXLeipzig","FrankfurtXStuttgart","HeidenheimXBochum","HoffenheimXMainz",
    "DarmstadtXKoln", "MgladbachXHoffenheim", "LeipzigXHeidenheim", "BochumXWolfsburg", "StuttgartXBremen", "MainzXFreiburg", "LeverkusenXDortmund", "AugsburgXFrankfurt", "BayernXUnion_Berlin",
    "HoffenheimXBochum", "HeidenheimXDarmstadt", "Union_BerlinXMgladbach", "FrankfurtXBayern", "BremenXAugsburg", "WolfsburgXFreiburg", "DortmundXLeipzig", "StuttgartXLeverkusen", "KolnXMainz",
    "MgladbachXBremen", "MainzXHeidenheim", "DarmstadtXWolfsburg", "AugsburgXDortmund", "BochumXUnion_Berlin", "LeipzigXHoffenheim", "FreiburgXKoln", "LeverkusenXFrankfurt","BayernXStuttgart",
    "BremenXLeipzig","DortmundXMainz","HoffenheimXDarmstadt","Union_BerlinXKoln","HeidenheimXFreiburg","LeverkusenXBochum","FrankfurtXMgladbach","StuttgartXAugsburg","WolfsburgXBayern",
    "BayernXHoffenheim","KolnXHeidenheim","MainzXWolfsburg","AugsburgXLeverkusen","LeipzigXFrankfurt","FreiburgXUnion_Berlin","DarmstadtXDortmund","BochumXBremen","MgladbachXStuttgart",
    "HeidenheimXWolfsburg","KolnXDortmund","DarmstadtXFrankfurt","FreiburgXHoffenheim","BochumXStuttgart","LeipzigXLeverkusen","BayernXBremen","MgladbachXAugsburg","MainzXUnion_Berlin",
    "FrankfurtXMainz","AugsburgXBayern","BremenXFreiburg","HoffenheimXHeidenheim","StuttgartXLeipzig","WolfsburgXKoln","LeverkusenXMgladbach","Union_BerlinXDarmstadt","DortmundXBochum",
    "HeidenheimXDortmund","MainzXBremen","DarmstadtXLeverkusen","BayernXMgladbach","FreiburgXStuttgart","BochumXAugsburg","KolnXFrankfurt","WolfsburgXHoffenheim","LeipzigXUnion_Berlin",
    "DortmundXFreiburg","Union_BerlinXWolfsburg","MgladbachXDarmstadt","FrankfurtXBochum","AugsburgXLeipzig","BremenXHeidenheim","LeverkusenXBayern","StuttgartXMainz","HoffenheimXKoln",
    "KolnXBremen", "HeidenheimXLeverkusen", "MainzXAugsburg", "DarmstadtXStuttgart", "HoffenheimXUnion_Berlin", "WolfsburgXDortmund", "LeipzigXMgladbach", "FreiburgXFrankfurt", "BochumXBayern",
    "LeverkusenXMainz", "Union_BerlinXHeidenheim", "MgladbachXBochum", "BremenXDarmstadt", "StuttgartXKoln", "BayernXLeipzig", "FrankfurtXWolfsburg", "DortmundXHoffenheim", "AugsburgXFreiburg",
    "FreiburgXBayern", "HeidenheimXFrankfurt", "Union_BerlinXDortmund", "MainzXMgladbach", "DarmstadtXAugsburg", "BochumXLeipzig", "WolfsburgXStuttgart", "KolnXLeverkusen", "HoffenheimXBremen",
    "StuttgartXUnion_Berlin", "MgladbachXKoln", "AugsburgXHeidenheim","BayernXMainz","LeipzigXDarmstadt","BremenXDortmund","BochumXFreiburg","FrankfurtXHoffenheim","LeverkusenXWolfsburg",
    "KolnXLeipzig","HeidenheimXMgladbach","Union_BerlinXBremen","MainzXBochum","DarmstadtXBayern","WolfsburgXAugsburg","HoffenheimXStuttgart","FreiburgXLeverkusen","DortmundXFrankfurt",
    "LeverkusenXHoffenheim","MgladbachXFreiburg","FrankfurtXUnion_Berlin","LeipzigXMainz","BremenXWolfsburg","BayernXDortmund","AugsburgXKoln","StuttgartXHeidenheim","BochumXDarmstadt",
    "FrankfurtXBremen", "HeidenheimXBayern", "KolnXBochum", "Union_BerlinXLeverkusen","MainzXDarmstadt","FreiburgXLeipzig","DortmundXStuttgart","HoffenheimXAugsburg","WolfsburgXMgladbach",
    "AugsburgXUnion_Berlin","MainzXHoffenheim", "MgladbachXDortmund", "BayernXKoln", "LeipzigXWolfsburg", "BochumXHeidenheim", "StuttgartXFrankfurt", "DarmstadtXFreiburg", "LeverkusenXBremen",
    "FrankfurtXAugsburg", "HeidenheimXLeipzig", "KolnXDarmstadt", "HoffenheimXMgladbach", "WolfsburgXBochum", "Union_BerlinXBayern", "BremenXStuttgart", "DortmundXLeverkusen", "FreiburgXMainz",
    "BochumXHoffenheim", "AugsburgXBremen", "BayernXFrankfurt", "LeipzigXDortmund", "FreiburgXWolfsburg", "LeverkusenXStuttgart", "MgladbachXUnion_Berlin", "MainzXKoln", "DarmstadtXHeidenheim",
    "HoffenheimXLeipzig", "DortmundXAugsburg", "BremenXMgladbach", "StuttgartXBayern", "WolfsburgXDarmstadt", "KolnXFreiburg", "Union_BerlinXBochum", "FrankfurtXLeverkusen", "HeidenheimXMainz",
    "AugsburgXStuttgart", "KolnXUnion_Berlin","MgladbachXFrankfurt", "LeipzigXBremen","FreiburgXHeidenheim", "MainzXDortmund", "DarmstadtXHoffenheim", "BayernXWolfsburg","BochumXLeverkusen",
    "HeidenheimXKoln","Union_BerlinXFreiburg","LeverkusenXAugsburg","DortmundXDarmstadt","FrankfurtXLeipzig","BremenXBochum","HoffenheimXBayern","StuttgartXMgladbach","WolfsburgXMainz"
]

# Dynamically create variables in globals
for i, game in enumerate(games_names):
    globals()[game] = dataframes_list[i]

# Now, safely create games_list
games_list = []
for game in games_names:
    if game in globals():
        games_list.append(globals()[game])
    else:
        raise NameError(f"Variable '{game}' not found in globals.")

# Add names to dataframes
for i, df in enumerate(games_list):
    df.name = games_names[i]



## This is the final function

In [13]:
def calculate_league_table():
    # Input validation and prompt for user input
    while True:
        try:
            minute_from = int(input("From which minute do you want goals to count? "))
            minute_to = int(input("To which minute do you want goals to count? "))
            if minute_from < 0 or minute_to < 0 or minute_from > minute_to:
                raise ValueError
            break
        except ValueError:
            print("Please enter valid integers where 'From' is less than or equal to 'To', and both are non-negative.")

    # Convert user inputs to boolean
    exclude_top_goalscorers = input("Do goals from top scorers count? (Yes/No): ").strip().lower() == 'no'
    exclude_most_valuable_players = input("Do goals from most valuable players count? (Yes/No): ").strip().lower() == 'no'
    exclude_captains = input("Do goals from captains count? (Yes/No): ").strip().lower() == 'no'
    exclude_substituted_in_scorers = input("Do goals from substituted players count? (Yes/No): ").strip().lower() == 'no'

    # Lists of topscorers, most valuable players, and captains
    top_goalscorers = pd.DataFrame(["H. Kane", "V. O. Boniface", "B. Šeško", "S. Guirassy", "M. Beier", "E. Demirović",
                                    "D. Malen", "M. Ducksch", "O. Marmoush", "J. Wind", "T. Kleindienst",
                                    "R. Hack", "J. Burkardt", "K. Stöger", "T. Skarke", "V. Grifo", "R. Gosens", "D. Selke"],
                                   columns=['goalscorers'])

    most_valuable_players = pd.DataFrame(["J. Musiala", "F. Wirtz", "S. Guirassy", "X. Simons", "J. Brnadt", "R. K. Muani",
                                          "M. Beier", "P. Wanner", "N. Füllkrug", "R. Doan", "E. Demirović", "L. Majer",
                                          "B. Gruda", "M. Koné", "D. Doekhi", "P. Osterhage", "J. Chabot", "T. Skarke"],
                                         columns=['players'])

    captains = pd.DataFrame(["M. Neuer", "E. Can", "W. Orbán", "C. Trimmel", "C. Günter", "L. Hradecky", "S. Rode",
                             "M. Arnold", "S. Widmer", "J. Omlin", "F. Kainz", "O. Baumann", "M. Friedl", "A. Losilla",
                             "E. Demirović", "W. Anton", "P. Mainka", "F. Holland"],
                            columns=['captains'])

    # Initialize league table and match count dictionaries
    league_table = {}
    match_count = {}

    for df in games_list:
        try:
            # Extract home and away team names from DataFrame name
            game_name = df.name
            if 'X' not in game_name:
                raise ValueError(f"Game name '{game_name}' is not in the correct format 'HomeTeamXAwayTeam'")

            home_team, away_team = game_name.split('X')

            # Initialize teams in the league table
            if home_team not in league_table:
                league_table[home_team] = {'Points': 0, 'G+': 0, 'G-': 0, 'W': 0, 'L': 0, 'D': 0}
                match_count[home_team] = 0
            if away_team not in league_table:
                league_table[away_team] = {'Points': 0, 'G+': 0, 'G-': 0, 'W': 0, 'L': 0, 'D': 0}
                match_count[away_team] = 0

            # Increment match count
            match_count[home_team] += 1
            match_count[away_team] += 1

            # Ensure necessary columns exist
            necessary_columns = ['Minute', 'Home Event', 'Away Event']
            if exclude_substituted_in_scorers:
                necessary_columns += ['Home Player In', 'Away Player In']

            missing_columns = [col for col in necessary_columns if col not in df.columns]

            if missing_columns:
                print(f"Missing columns: {missing_columns}. Assigning default values.")
                for col in missing_columns:
                    df[col] = None

            # Ensure scorer columns exist
            df['Home Scorer'] = df.get('Home Scorer', pd.Series([None] * len(df)))
            df['Away Scorer'] = df.get('Away Scorer', pd.Series([None] * len(df)))

            # Filter events to include only those within the specified minute range
            df_filtered = df[(df['Minute'] >= minute_from) & (df['Minute'] <= minute_to)]

            # Apply conditions for excluding goals
            if exclude_top_goalscorers:
                df_filtered = df_filtered[~df_filtered['Home Scorer'].isin(top_goalscorers['goalscorers']) &
                                          ~df_filtered['Away Scorer'].isin(top_goalscorers['goalscorers'])]

            if exclude_most_valuable_players:
                df_filtered = df_filtered[~df_filtered['Home Scorer'].isin(most_valuable_players['players']) &
                                          ~df_filtered['Away Scorer'].isin(most_valuable_players['players'])]

            if exclude_substituted_in_scorers:
                home_subs_in = df_filtered['Home Player In'].dropna().unique()
                away_subs_in = df_filtered['Away Player In'].dropna().unique()
                df_filtered = df_filtered[~df_filtered['Home Scorer'].isin(home_subs_in) &
                                          ~df_filtered['Away Scorer'].isin(away_subs_in)]
            if exclude_captains:
                df_filtered = df_filtered[~df_filtered['Home Scorer'].isin(captains['captains']) &
                                          ~df_filtered['Away Scorer'].isin(captains['captains'])]

            # Calculate goals for home and away teams
            home_goals_count = df_filtered[df_filtered['Home Event'] == 'Goal'].shape[0]
            away_goals_count = df_filtered[df_filtered['Away Event'] == 'Goal'].shape[0]

            # Update goals scored and conceded
            league_table[home_team]['G+'] += home_goals_count
            league_table[home_team]['G-'] += away_goals_count
            league_table[away_team]['G+'] += away_goals_count
            league_table[away_team]['G-'] += home_goals_count

            # Update points and statistics based on match result
            if home_goals_count > away_goals_count:
                league_table[home_team]['Points'] += 3
                league_table[home_team]['W'] += 1
                league_table[away_team]['L'] += 1
            elif home_goals_count < away_goals_count:
                league_table[away_team]['Points'] += 3
                league_table[away_team]['W'] += 1
                league_table[home_team]['L'] += 1
            else:
                league_table[home_team]['Points'] += 1
                league_table[away_team]['Points'] += 1
                league_table[home_team]['D'] += 1
                league_table[away_team]['D'] += 1

        except Exception as e:
            print(f"Error processing dataframe {df.name}: {e}")

    # Convert league table to a DataFrame
    league_df = pd.DataFrame.from_dict(league_table, orient='index')
    league_df['+/-'] = league_df['G+'] - league_df['G-']
    league_df['Matches'] = league_df.index.map(match_count)

    # Sort the league table
    league_df = league_df.sort_values(by=['Points', '+/-', 'G+'], ascending=False)
    league_df.reset_index(inplace=True)
    league_df.rename(columns={'index': 'Team'}, inplace=True)

    # Add rank with conditions
    league_df["#"] = league_df.index + 1  # Adding 1 to make it 1-based index

    # Known final positions
    table_without_conditions = ["Leverkusen", "Stuttgart", "Bayern", "Leipzig", "Dortmund", "Frankfurt",
                                "Hoffenheim", "Heidenheim", "Bremen", "Freiburg", "Augsburg", "Wolfsburg",
                                "Mainz", "Mgladbach", "Union_Berlin", "Bochum", "Koln", "Darmstadt"]

    # Create DataFrame for known final positions
    known_positions = pd.DataFrame(table_without_conditions, columns=['Team'])
    known_positions['Rank_without_conditions'] = known_positions.index + 1  # Adding 1 to make it 1-based index

    # Merge calculated league table with known final positions
    merged_df = pd.merge(league_df, known_positions, on='Team', how='left')

    # Calculate position differences
    merged_df['Change'] = merged_df['Rank_without_conditions'] - merged_df["#"]
    merged_df['Change'] = merged_df['Change'].apply(lambda x: f"+{x}" if x > 0 else ("=" if x == 0 else str(x)))

    # Reset index to avoid displaying it
    merged_df.reset_index(drop=True, inplace=True)

    merged_df[""] = ""

    # Return DataFrame without index
    return merged_df[["#", "Change", "Team", "", "", "", "", "", "Matches", "W", "D", "L", "G+", "G-", "+/-", "Points"]].style.hide(axis='index')


In [14]:
calculate_league_table()

#,Change,Team,,,,,,Matches,W,D,L,G+,G-,+/-,Points
1,=,Leverkusen,,,,,,34,28,6,0,89,24,65,90
2,+1,Bayern,,,,,,34,24,2,8,94,41,53,74
3,-1,Stuttgart,,,,,,34,23,4,7,78,38,40,73
4,=,Leipzig,,,,,,34,20,7,7,77,38,39,67
5,=,Dortmund,,,,,,34,19,7,8,66,42,24,64
6,=,Frankfurt,,,,,,34,12,13,9,50,48,2,49
7,=,Hoffenheim,,,,,,34,14,6,14,66,64,2,48
8,+1,Bremen,,,,,,34,11,9,14,47,54,-7,42
9,+1,Freiburg,,,,,,34,11,9,14,44,57,-13,42
10,-2,Heidenheim,,,,,,34,10,11,13,48,53,-5,41
